In [ ]:
!pip install -U pyspellchecker
!pip install emoji

Requirement already up-to-date: pyspellchecker in /usr/local/lib/python3.7/dist-packages (0.6.2)


# Tutorial de NLTK para NLP

En este corto tutorial veremos algunas de las herramientas y métodos que tiene NLTK para el procesamiento de texto. También nos apoyaremos en la librería [RE](https://docs.python.org/3/library/re.html) (Expresiones Regulares) para la limpieza del texto. Empezamos creando una lista de Tweets sintéticos, ya que lo que nos interesa en este Notebook es la parte del procesamiento. 


In [ ]:
import re
from nltk.tokenize import word_tokenize, TweetTokenizer # tokenizador
from string import punctuation 
from nltk.corpus import stopwords # elimina palabras de poco valor
from nltk.stem.wordnet import WordNetLemmatizer # stemming lemming
# Instalamos algunas de las APIs y herramientas de NLTK
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
# Creamos nuestras stopwords
stopwords_ = set(stopwords.words('english') + list(punctuation)+ ["rt"]) 

También podemos eliminar alguna palabra de la lista de Stopwords. Qué pasaba con las negaciones en el caso de uso pasado???

In [ ]:
print('Esta la palabra "no"? ->','no' in stopwords_)
stopwords_.remove('no')
print('Y ahora??? ->','no' in stopwords_)

Esta la palabra "no"? -> True
Y ahora??? -> False


In [ ]:
tweets = [
    "Hellooooooo!! WHat's wrong with the world today!!!! @user1 #bored #world",
    "RT I simply looove this outfit www.fakeoutfitpage.com, it's my favorite. I can't stop thinking about it!! @shopingmall #fashion",
    "The launch of the @SpaceY has been cancelled, I don't want to believe this :( . #notcool #spacey",
    "RT Hey! Visit www.thisisafakeurl.com to get a FREEEEE coupon on ALL the @fakeshop productos. #Freecoupon",
    "What a nice album @hotplay has released, I JUST CAN'T STOP LISTENING TO IT. THE BEEEEEEEEEST!! #hotplay #music"    
    
]

### Vamos a trabajar con un Tweet para revisar cada paso, luego creamos una función para realizarlo en todo el corpus

In [ ]:
# Vamos a trabajar con un tweet para ir viendo cada paso
tweet = tweets[1]
tweet

" RT I simply looove this outfit www.fakeoutfitpage.com, it's my favorite. I can't stop thinking about it!! @shopingmall #fashion"

In [ ]:
# empezamos transformando a minúsculas
tweet = tweet.lower()
tweet

" rt i simply looove this outfit www.fakeoutfitpage.com, it's my favorite. i can't stop thinking about it!! @shopingmall #fashion"

In [ ]:
# removemos la URL usando expresiones regulares
tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet)
tweet

" rt i simply looove this outfit URL it's my favorite. i can't stop thinking about it!! @shopingmall #fashion"

In [ ]:
"@usuario".replace("@","")

'usuario'

In [ ]:
# removemos el usario expresiones regulares
tweet = re.sub('@[^\s]+', 'USUARIO', tweet)
tweet

" rt i simply looove this outfit URL it's my favorite. i can't stop thinking about it!! USUARIO #fashion"

In [ ]:
# removemos el hashtag
tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
tweet

" rt i simply looove this outfit URL it's my favorite. i can't stop thinking about it!! USUARIO fashion"

In [ ]:
# Eliminamos la repetición de caracteres en LOOOOOOOOOOVE
tweet = re.sub(r'(.)\1+', r'\1\1', tweet) 

In [ ]:
# probamos los tokenizadores que cargamos 
# word tokenizer 
print(word_tokenize(tweet)) # remove repeated characters (helloooooooo into hello)

['rt', 'i', 'simply', 'loove', 'this', 'outfit', 'URL', 'it', "'s", 'my', 'favorite', '.', 'i', 'ca', "n't", 'stop', 'thinking', 'about', 'it', '!', '!', 'USUARIO', 'fashion']


No me gusta mucho el resultado ya que palabras como it's se dividen, igual can't. Probamos el otro

In [ ]:
tknzr = TweetTokenizer()
print(tknzr.tokenize(tweet))

['rt', 'i', 'simply', 'loove', 'this', 'outfit', 'URL', "it's", 'my', 'favorite', '.', 'i', "can't", 'stop', 'thinking', 'about', 'it', '!', '!', 'USUARIO', 'fashion']


Este es mejor, y además está diseñado para Twitter :). Lo utilizaremos de ahora en adelante 

In [ ]:
# TOkenizamos 
tweet = tknzr.tokenize(tweet)
print(tweet)

['rt', 'i', 'simply', 'loove', 'this', 'outfit', 'URL', "it's", 'my', 'favorite', '.', 'i', "can't", 'stop', 'thinking', 'about', 'it', '!', '!', 'USUARIO', 'fashion']


In [ ]:
# Quitamos las stopwords
tweet = [word for word in tweet if word not in stopwords_] # nuestras stopwords
print(tweet)

['simply', 'loove', 'outfit', 'URL', 'favorite', "can't", 'stop', 'thinking', 'USUARIO', 'fashion']


In [ ]:
def stem_lemm(tweet_list):
    """
    Lemmatiza un tweeet uitlizando el lematizador de NLTK
    :param tweet_list: Un tweet tokenizado previamente
    :return: Una lista de tokens
    """
    lem = WordNetLemmatizer()
    normalized_tweet = []
    for word in tweet_list:
        normalized_text = lem.lemmatize(word,'v')
        normalized_tweet.append(normalized_text)
    return normalized_tweet

print(stem_lemm(tweet))

['simply', 'loove', 'outfit', 'URL', 'favorite', "can't", 'stop', 'think', 'USUARIO', 'fashion']


## Corrección gramatical 

En muchos casos también es posible que necesitemos añadir un corrector gramatical para nuestro proyecto. Eso se puede hacer mediante varias librerías, pero una de las más comunes es SpellChecker


In [ ]:
from spellchecker import SpellChecker

spell = SpellChecker()

# Encuentra las que tienen errores
misspelled = spell.unknown(tweet)

for word in misspelled:
    # La correción más probable
    print(spell.correction(word))

    # otros conadidatos
    print(spell.candidates(word))

love
{'love', 'looe', 'loose'}
usuario
{'usuario'}
usl
{'curl', 'ural', 'ura', 'uri', 'urn', 'crl', 'usl', 'ucl', 'rl', 'ure', 'urc', 'uhl', 'urd', 'erl', 'hurl', 'purl', 'brl', 'ur', 'ual', 'urr', 'u/l'}


In [ ]:
# Corregimos el tweet. 
[spell.correction(word) if word in misspelled else word for word in tweet]

['simply',
 'love',
 'outfit',
 'URL',
 'favorite',
 "can't",
 'stop',
 'thinking',
 'USUARIO',
 'fashion']

# Todo junto 

Ahora que hemos visto y probado varios métodos. Podemos hacer una función o una clase con todo lo necesario para limpiar nuestros Tweets

In [ ]:
def limpia_tweets(tweet): 
    # empezamos transformando a minúsculas
    clean_tweet = tweet.lower()
    # removemos la URL usando expresiones regulares
    clean_tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', '', clean_tweet) # ya no reemplazamos por URL, sino por vacio
    # Quitamos los usuarios
    clean_tweet = re.sub('@[^\s]+', '', clean_tweet)
    # Quitamos hashtag
    clean_tweet = re.sub(r'#([^\s]+)', r'\1', clean_tweet)
    # Eliminamos repetición de caracteres 
    clean_tweet = re.sub(r'(.)\1+', r'\1\1', clean_tweet) 
    # Tokenizamos 
    clean_tweet = tknzr.tokenize(clean_tweet)
    # Quitamos las stopwords
    clean_tweet = [word for word in clean_tweet if word not in stopwords_] # nuestras stopwords
    # lemming y stemming 
    clean_tweet = stem_lemm(clean_tweet)
    # Encuentra las que tienen errores
    misspelled = spell.unknown(clean_tweet)
    # Corregimos el tweet. 
    clean_tweet = [
        spell.correction(word) if word in misspelled else word for word in clean_tweet]
    return clean_tweet
    


In [ ]:
print(limpia_tweets(tweets[1]))

['simply', 'love', 'outfit', 'favorite', "can't", 'stop', 'think', 'fashion']


In [ ]:
# Todos nuestros Tweets
tweets_limpios = [limpia_tweets(tweet) for tweet in tweets]

In [ ]:
for t in tweets_limpios:
    print(t)

['hello', 'whats', 'wrong', 'world', 'today', 'bore', 'world']
['simply', 'love', 'outfit', 'favorite', "can't", 'stop', 'think', 'fashion']
['launch', 'cancel', 'want', 'believe', 'of', 'notcool', 'spacey']
['hey', 'visit', 'get', 'free', 'coupon', 'products', 'freecoupon']
['nice', 'album', 'release', "can't", 'stop', 'listen', 'best', 'hotly', 'music']


# Emojis!!

Para el tratamiento de emojis, comparto dos alternativas. Una es traducirlo (requiere instalar la librería emoji) o eliminarlos con expresiones regulares.

In [ ]:
import emoji #!pip install emoji

# Traducimos usando la librería emoji
print(emoji.demojize('Python is 👍'))

Python is :thumbs_up:


In [ ]:
def remove_emoji(string):
    """
    Elimina los emojis de una cadena de texto. 
    :param string: Una cadena de texto 
    :return: Una cadena de texto sin los emojis
    
    """
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

print(remove_emoji('Python is 👍'))

Python is 
